In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)

df = pd.read_csv('jeopardy.csv')
df.rename(columns={
    'Show Number' : 'show_number', ' Air Date' : 'air_date',
    ' Round' : 'jeopardy_round',
    ' Category' : 'category',
    ' Value' : 'stakes',
    ' Question' : 'question',
    ' Answer' : 'answer'}, inplace=True
)
df['stakes'] = df['stakes'].str.replace('\$', '').str.replace(',','').str.replace('None','0').astype(float)

<ipython-input-2-0491b77a8b88>:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df['stakes'] = df['stakes'].str.replace('\$', '').str.replace(',','').str.replace('None','0').astype(float)


In [3]:
#Define a function that takes in a list of search words and a dataframe and returns the rows in which the question contains all of those words
def question_search(word_list, dataframe):
    results = df
    for word in word_list:
        results = results.loc[df['question'].str.contains(word, case=False)]
        print(results.info())
        
    return results
    
print(question_search(['King'], df).head(3))
#print(question_search(['King'], df).stakes.mean())
king_questions = question_search(['King'], df)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 7409 entries, 34 to 216916
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   show_number     7409 non-null   int64  
 1   air_date        7409 non-null   object 
 2   jeopardy_round  7409 non-null   object 
 3   category        7409 non-null   object 
 4   stakes          7409 non-null   float64
 5   question        7409 non-null   object 
 6   answer          7409 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 463.1+ KB
None
    show_number    air_date    jeopardy_round                    category  \
34         4680  2004-12-31  Double Jeopardy!                 "X"s & "O"s   
40         4680  2004-12-31  Double Jeopardy!  DR. SEUSS AT THE MULTIPLEX   
50         4680  2004-12-31  Double Jeopardy!  DR. SEUSS AT THE MULTIPLEX   

    stakes  \
34   400.0   
40  1200.0   
50  2000.0   

                                                   

In [76]:
#Write a function that returns the count of the unique answers to all of the questions in a dataset

def count_unique(dataframe):
    print(dataframe.answer.nunique(), 'unique answers')
count_unique(king_questions)
    

5268 unique answers


In [131]:
#How many questions from the 90s use the word "Computer" compared to questions from the 2000s?

def decade_compare(word, dataframe):
    dataframe90s = dataframe.loc[dataframe['air_date'].str.contains('199')]
    dataframe00s = dataframe.loc[dataframe['air_date'].str.contains('200')]
    total90s = dataframe90s.shape[0]
    total00s = len(dataframe90s.index)
    dataframe90s_word = dataframe90s.loc[dataframe90s['question'].str.contains(word, case=False)]
    dataframe00s_word = dataframe00s.loc[dataframe00s['question'].str.contains(word, case=False)]
    
    percent90s =  dataframe90s_word.shape[0] / float(total90s)
    percent00s = len(dataframe00s_word.index) / float(total00s)
    
    print('Word: ', word, '90s: ', percent90s, '00s: ', percent00s)

decade_compare('Teflon', df)

Word:  Teflon 90s:  5.286809410520751e-05 00s:  8.811349017534584e-05


In [1]:
#Are you more likely to find certain categories, like "Literature" in Single Jeopardy or Double Jeopardy
def count_unique_cats(dataframe):
    print(dataframe.category.nunique(), 'unique categories')
#count_unique_cats(df)

#find the N most common categories. start with one:
#method1: loop over all 200k records, looping each time through all 200k, to add up the number of instances of each category (slow)
#method2: create a list of lists to put (category, num_of). go through all 200k records, adding categories or adding a +1 to the num of cats.
#method3: google for useful pandas functions
#method4: list comprehensions? looks like that won't work https://stackoverflow.com/questions/16476924/how-to-iterate-over-rows-in-a-dataframe-in-pandas
#method5: groupby operation
#def find_common_cats2(number_of_cats, dataframe):
#    list_of_cats = []
##    for row in dataframe:
#        list_of_cats.append(dataframe['category'])
#    print(list_of_cats[0])
#print(df.head(5))
def find_common_cats5(num_of_cats, dataframe):
    #print(dataframe.groupby('category').aggregate(np.size))
    #print(dataframe.groupby('category').apply(lambda x: np.size(x) > 10))
    #print(dataframe.groupby(['category', 'jeopardy_round']).apply(lambda x: np.size(x) > 35)) #what is this lambda function measuring?
    newdf = dataframe.groupby(['category', 'jeopardy_round']).size().reset_index()
    newdf.rename(columns={0:'count_of'}, inplace=True)
    newdf = newdf.sort_values('count_of', ascending=False).reset_index()
    
    #in order to get 'literature' DJ and SJ next to each other, could do a groupby on just category, then sort it by count, then export 
    #category as a list. Then could use that list as an index, or sort by it or something, against the df grouped by cat and jeo.
    
    #or, could rearrange so that double jep has its own column. to do this, create two separate dataframes, then do a merge.
    single_jeo_df = newdf.loc[newdf['jeopardy_round'] == 'Jeopardy!']
    single_jeo_df.rename(columns={'count_of':'count_of_single_jep'}, inplace=True)
    single_jeo_df.drop(['jeopardy_round', 'index'], inplace=True, axis=1)
    double_jeo_df = newdf.loc[newdf['jeopardy_round'] == 'Double Jeopardy!']
    double_jeo_df.rename(columns={'count_of':'count_of_double_jep'}, inplace=True)
    double_jeo_df.drop(['jeopardy_round', 'index'], inplace=True, axis=1)

    
    reshaped_df = pd.merge(single_jeo_df, double_jeo_df, how='outer')
    
    #print(newdf.head(20))
    print(reshaped_df.head(50))
    print(reshaped_df.tail(10))
    
#find_common_cats5(3, df)
    
    
    


In [4]:
#continued, with method6 from https://stackoverflow.com/questions/35236213/how-to-compare-group-sizes-in-pandas

#results = df.groupby(['category', 'jeopardy_round']).agg({'question':pd.Series.count})
#print(results)

                                              question
category                    jeopardy_round            
 A JIM CARREY FILM FESTIVAL Jeopardy!                5
"!"                         Jeopardy!                5
"-ARES"                     Double Jeopardy!         5
"-ICIAN" EXPEDITION         Jeopardy!                5
"...OD" WORDS               Double Jeopardy!         5
...                                                ...
“R” MOVIES                  Double Jeopardy!         5
“SAINTS”                    Double Jeopardy!         4
“SOUTH”                     Double Jeopardy!         5
“STREETS”                   Jeopardy!                5
“WH”AT IS IT?               Double Jeopardy!         5

[31686 rows x 1 columns]


In [ ]:
#Grab random questions, and use the input function to get a response from the user. 
#Check to see if that response was right or wrong.
#1. select a question at random, print it. 2. get input. 3. compare input to answer, determine correctness. 4. output correctness
score = 0
keep_playing = 'y'
while keep_playing == 'y':
    random_question_row = df.sample()
    random_question = random_question_row.question.values[0]
    random_answer = random_question_row.answer.values[0]

    print(random_question_row['question'].values[0])
    user_response = input('--> ')


    if user_response.lower() == random_answer.lower():
        print('correct! ', random_question_row.stakes.values[0], 'points for you.')
        score = score + random_question_row.stakes.values[0]
    else:
        print("I'm sorry that's incorrect. You've lost ", random_question_row.stakes.values[0], 'points.')
        score = score - random_question_row.stakes.values[0]
    keep_playing = input('keep playing? Y/N: ').lower()

print('Thanks for playing! your final score was ', score)

This pair has been selling books in stores "Since 1873" & on the internet since 1997
--> Simon and Schuster
I'm sorry that's incorrect. You've lost  400.0 points.
keep playing? Y/Ny
On Dec. 23, 1921 President Harding pardoned this Socialist so that he could have Christmas dinner with his wife
--> Turing
I'm sorry that's incorrect. You've lost  500.0 points.
keep playing? Y/Ny
Beverly Cleary wrote several books about Beatrice, called Beezus, & this younger sister of hers
--> Boobo
I'm sorry that's incorrect. You've lost  800.0 points.
keep playing? Y/Ny
First produced in 1920, this Shaw comedy takes place in the unconventional home of elderly Capt. Shotover
--> old man
I'm sorry that's incorrect. You've lost  1000.0 points.
